In [106]:
%load_ext autoreload
%autoreload 2


import numpy as np
from utils.data_helper import *
from utils.data import *
from utils.stats import *
from utils.performance import *
from plotly.subplots import make_subplots
from account import Binance
import pandas as pd
import warnings
from strategy_v3.Strategy import *
from strategy_v3.Executor import ExecutorBinance, ExecutorBacktest
from strategy_v3.ExecuteSetup import *
from strategy_v3.DataLoader import DataLoaderBinance
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from time import sleep
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
symbol = 'BTCFDUSD'
binance = Binance()
df = binance.get_historical_instrument_price(symbol, interval='1m', start_str='2 hours ago')
plot_price_ohcl(df, symbol)
df['std'] = df['Close'].rolling(15).std()

# Market Making

In [296]:
symbol = 'BTCFDUSD'
df = Binance().get_historical_instrument_price(symbol, interval='1m', start_str='2 hours ago')
df['std'] = df['Close'].rolling(15).std()
df['adv'] = df['Volume'].rolling(15).mean()
vol = df['std'].iloc[-1]
adv = df['adv'].iloc[-1]

df_bid, df_ask = Binance().get_order_book(instrument=symbol, limit=1000)
df_trades_bid, df_trades_ask = Binance().get_aggregate_trades(instrument=symbol, start_date='10 seconds ago')
df_trades = pd.concat([df_trades_bid, df_trades_ask])

best_bid = df_bid.iloc[0]['price']
best_ask = df_ask.iloc[0]['price']
mid_px = (best_bid + best_ask) / 2
spread_current = best_ask - best_bid
position_size = 100/mid_px

# spread based on ADV
flow_target = 0.02 * adv
spread_ask = df_ask[df_ask['quantity_cum'] > flow_target/2]['price'].min()
spread_bid = df_bid[df_bid['quantity_cum'] > flow_target/2]['price'].max()        
spread = spread_ask - spread_bid

# Arrival rate
ar_bid = df_trades_bid['quantity'].sum()/10
ar_ask = df_trades_ask['quantity'].sum()/10
ar_skew = ar_ask - ar_bid

# Volume weighted mid price
# v1
# average VWAP by ask and VWAP by bids
vw_bid = (df_trades_bid['price'] * df_trades_bid['quantity']).sum() / df_trades_bid['quantity'].sum()
vw_ask = (df_trades_ask['price'] * df_trades_ask['quantity']).sum() / df_trades_ask['quantity'].sum()
vwmp = (vw_ask + vw_bid) / 2

# v2
# volume weighted average of all orders
vwmp = (df_trades['price'] * df_trades['quantity']).sum() / df_trades['quantity'].sum()

# v3
# skew in arrival
ar_skew_sum = ar_skew * 10
if ar_skew > 0:
    vwmp_skew = df_ask[df_ask['quantity_cum'] > abs(ar_skew_sum)].iloc[0]['price'] - best_ask
else:
    vwmp_skew = df_bid[df_bid['quantity_cum'] > abs(ar_skew_sum)].iloc[0]['price'] - best_bid

vwmp = mid_px + vwmp_skew
skew = vwmp - mid_px

current_position = -0.0015*0
target_position = 0
vol = df['std'].iloc[-1] 
gamma = 0.1

r = vwmp - (current_position - target_position) * gamma * vol ** 2
order_bid = min(r - spread/2, best_bid)
order_ask = max(r + spread/2, best_ask)
new_spread = order_ask - order_bid

print(f'current spread = {spread_current:.4f}, target spread = {new_spread:.4f}, adv = {adv:.4f}')
print(f'vwmp = {vwmp:.4f}, mid = {mid_px:.4f}, skew = {skew:.4f}')
print(f'arrival bid = {ar_bid:.4f}, arrival ask = {ar_ask:.4f}, arrival_skew = {ar_skew:.4f}')   

fig = make_subplots(rows=2, subplot_titles=['LOB Depth', 'Trades Executed (10 seconds ago)'], vertical_spacing=0.1)
fig.update_layout(title=symbol, width=800, height=1000, hovermode='x')
fig.add_trace(go.Scatter(x=df_bid['price'], y=df_bid['quantity_cum'], name='bid'),row=1, col=1)
fig.add_trace(go.Scatter(x=df_ask['price'], y=df_ask['quantity_cum'], name='ask'),row=1, col=1)
fig.add_vline(order_bid, line_dash="dash", line_color="blue",row=1, col=1)
fig.add_vline(order_ask, line_dash="dash", line_color="red",row=1, col=1)
fig.add_vline(vwmp, line_dash="dash", line_color="green",row=1, col=1)
fig.add_vline(r, line_dash="dash", line_color="black",row=1, col=1)

colors = plotly.colors.DEFAULT_PLOTLY_COLORS

fig.add_trace(go.Bar(x=df_trades_bid['price']//2*2, y=df_trades_bid['quantity'], name='bid', marker={'color': colors[0]}), row=2,col=1)
fig.add_trace(go.Bar(x=df_trades_ask['price']//2*2, y=df_trades_ask['quantity'], name='ask', marker={'color': colors[1]}), row=2,col=1)
fig.add_vline(mid_px, line_dash="dash", line_color="black",row=2, col=1)

current spread = 5.7800, target spread = 24.7150, adv = 29.8584
vwmp = 64985.1200, mid = 64970.9100, skew = 14.2100
arrival bid = 0.1190, arrival ask = 0.2315, arrival_skew = 0.1125


# Strategy performance

In [338]:
strategy = SimpleMarketMakingStrategy(
    instrument = 'BTCFDUSD',
    interval = '1m',
    vol_lookback = 20,                      
    spread_adv_factor = 0.05,    
    target_position = 0,                
    position_size = 100,
    price_decimal = 2,
    qty_decimal = 4,                             
    status = STATUS.RUN,
    verbose = True,       
    start_date = '2024-03-20 14:00:00'
)
strategy.set_data_loder(DataLoaderBinance())
strategy.set_executor(ExecutorBinance())
strategy.set_strategy_id('SMM_BTCv5')    

In [339]:
# pnl cutoff as 00:00 HKT
date = datetime.today()
date = datetime(year=date.year, month=date.month, day=date.day, tzinfo=ZoneInfo("HongKong"))
date = date.astimezone(ZoneInfo("UTC"))
date_str = date.strftime('%Y-%m-%d %H:%M:%S')

strategy.load_data(date_str)
strategy.summary()

,Measure,smm_SMM_BTCv5
0,Pnl,0.636359
1,Trading Fee,0.000000
2,Cumulative Return,1.003182
3,Annualized Return,20.618038
4,Annualized Volatility,0.501807
5,Annualized Sharpe Ratio,40.987530
6,Maximum Drawdown,-0.004541


# Analysis on strategy Log

In [340]:
df = strategy.get_log_data()
df = df.sort_values('date')
df = df.tail(300)
df['px_change'] = df['mid_price'].diff().fillna(0)
df['px_change_next'] = df['px_change'].shift(-1).fillna(0)

In [341]:
xcol = 'px_change'
a1, b1 = np.polyfit(df[xcol].to_numpy(), df['ar_skew'].to_numpy(), 1)
a2, b2 = np.polyfit(df[xcol].to_numpy(), df['skew'].to_numpy(), 1)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        f'Arrival Rate Skew vs {xcol}',     
        f'VWMP Skew vs {xcol}',             
    ],     
)
fig.update_layout(
    title=symbol,
    width=1000, height=500,
    showlegend=False,
    xaxis_title=xcol,
    xaxis2_title=xcol,
    yaxis_title="Arrival Rate Skew",    
    yaxis2_title="VWMP Skew",
)

colors = plotly.colors.DEFAULT_PLOTLY_COLORS
fig.add_trace(go.Scatter(x=df[xcol], y=df['ar_skew'], mode='markers', marker=dict(color=colors[0])), row=1, col=1)
fig.add_trace(go.Scatter(x=df[xcol], y=(a1*df[xcol] + b1), marker=dict(color=colors[0]), mode='lines'), row=1, col=1)             

fig.add_trace(go.Scatter(x=df[xcol], y=df['skew'], mode='markers', marker=dict(color=colors[1])), row=1, col=2)
fig.add_trace(go.Scatter(x=df[xcol], y=(a2*df[xcol] + b2), marker=dict(color=colors[1]), mode='lines'), row=1, col=2)             